<a href="https://colab.research.google.com/github/Tom-JD/Financial-Sentiment-Analysis-Using-NLP-finbert-/blob/main/ObtainingStockData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Obtaining the Company Announcement Data**

As mentioned in the other document, we will be scraping company announcements for Eurosports Global Ltd. Here is a general code I created to scrape them off the appropriate link into my Google drive to save me the time and manpower. It also served useful when I tested my code using other companys' announcements.

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive; drive.mount('/content/gdrive', force_remount=True)
%cd '/content/gdrive/My Drive/Colab Notebooks/Data'

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/Data


In [ ]:
drive.flush_and_unmount()

NameError: ignored

We use selenium drivers to download it. The SGX website repeatedly locked me out of it, denying requests if I ran the code too often, and the below arguments added to the webdriver aid in overcoming this.

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 904 kB 23.4 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.ne

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


Now we simply insert our url.

In [ ]:
url="https://sginvestors.io/sgx/stock/5g1-eurosports-gbl/company-announcement"

driver.get(url)

In [ ]:
driver.Url = "https://www.sgx.com/securities/company-announcements?value=EUROSPORTS%20GLOBAL%20LIMITED&type=company&from=20210101&to=20210824"

In [ ]:
#Had to pause for a while to avoid the website from detecting us
import time 
time.sleep(5)

#The main url contains a lot of subsidiary urls - we will collect all the links, then download the pdfs on that page.
elemLs=[]
elems = driver.find_elements_by_xpath("//a[@href]")
print(driver.current_url)
for elem in elems:
    x=elem.get_attribute("href")
    #print(x)
    elemLs.append(x)
    

print(elemLs)

#luckily only urls that contain 'links' has pdfs, so only keeping links that contain pdfs
import re
elemLs = [x for x in elemLs if re.match(r'https://links.sgx.com',x)]

print(driver.current_url)
#for each link, we download the pdfs.
import urllib.request

print(url)

for link in elemLs:
    time.sleep(5)
    driver.get(link)
    pdfLs=[]
    elems = driver.find_elements_by_xpath("//a[@href]")
    for elem in elems:
        x=elem.get_attribute("href")
        pdfLs.append(x)

    labels = driver.find_elements_by_xpath('/html/body/form/div[3]/div/div[2]/div[3]/dl/dd[2]') #finds time and date
    n=1
    for pdf in pdfLs: 
        #instead of accessing the website as user agent, which eventually results in a website ban, we have to temporarily open a new browser to access it which overcomes the problem
        opener = urllib.request.build_opener()
        opener.addheaders = [('User-agent', 'Mozilla/5.0')]
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(pdf, labels[0].text.replace(':','')+'_'+str(n)+'.pdf')
        n+=1

https://sginvestors.io/sgx/stock/5g1-eurosports-gbl/company-announcement
['https://sginvestors.io/', 'https://sginvestors.io/', 'https://sginvestors.io/', 'https://sginvestors.io/sgx/stock/5g1-eurosports-gbl/company-announcement#', 'https://sginvestors.io/market/ftse-st-indices-straits-times-index-constituents', 'https://sginvestors.io/market/sgx-market-information/', 'https://sginvestors.io/market/sgx-share-price-performance/straits-times-index-constituents', 'https://sginvestors.io/market/sgx-top-volume', 'https://sginvestors.io/market/sgx-top-value', 'https://sginvestors.io/market/sgx-top-gainers-by-percent', 'https://sginvestors.io/market/sgx-top-gainers-by-value', 'https://sginvestors.io/market/sgx-top-losers-by-percent', 'https://sginvestors.io/market/sgx-top-losers-by-value', 'https://sginvestors.io/market/sgx-breakout-price-3-month-high-volume-above-average', 'https://sginvestors.io/market/sgx-breakout-price-3-month-low-volume-above-average', 'https://sginvestors.io/market/sgx-

IndexError: ignored

In [ ]:
%cp -av "/content" "/content/drive/MyDrive/Colab Notebooks/Data"


'/content' -> '/content/drive/MyDrive/Colab Notebooks/Data/content'
'/content/.config' -> '/content/drive/MyDrive/Colab Notebooks/Data/content/.config'
'/content/.config/.last_opt_in_prompt.yaml' -> '/content/drive/MyDrive/Colab Notebooks/Data/content/.config/.last_opt_in_prompt.yaml'
'/content/.config/.last_survey_prompt.yaml' -> '/content/drive/MyDrive/Colab Notebooks/Data/content/.config/.last_survey_prompt.yaml'
'/content/.config/.last_update_check.json' -> '/content/drive/MyDrive/Colab Notebooks/Data/content/.config/.last_update_check.json'
'/content/.config/active_config' -> '/content/drive/MyDrive/Colab Notebooks/Data/content/.config/active_config'
'/content/.config/config_sentinel' -> '/content/drive/MyDrive/Colab Notebooks/Data/content/.config/config_sentinel'
'/content/.config/configurations' -> '/content/drive/MyDrive/Colab Notebooks/Data/content/.config/configurations'
'/content/.config/configurations/config_default' -> '/content/drive/MyDrive/Colab Notebooks/Data/content/.